In [1]:
import pandas as pd

In [3]:
train_data = pd.read_csv("../data/bank_churn_competition/train.csv")

In [12]:
for column in list(train_data.columns):
    # if column != "id": 
    print(f"{column}: {train_data[column].nunique()}\n")

id: 165034

CustomerId: 23221

Surname: 2797

CreditScore: 457

Geography: 3

Gender: 2

Age: 71

Tenure: 11

Balance: 30075

NumOfProducts: 4

HasCrCard: 2

IsActiveMember: 2

EstimatedSalary: 55298

Exited: 2

